In [1]:
%matplotlib widget
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
import seaborn as sns
import pandas as pd
import itertools
from pathlib import Path
data_dir = Path('data')

In [2]:
sns.set_style("whitegrid")


In [3]:
from script import Evolve, Torus, Ring, utilities, Plotter
from script.metric import *

In [4]:
n= 256
c_type= ConfigType.Ring
size = n
p_state= 0.5
seed= 654798203
steps=2000
np.random.seed(seed)
pi=np.random.permutation(n)
metricList= [Metric.SpinGlass, Metric.Magnetization]
magnetList = [0]
samples = 100

agree= {'init_magnet': [], 'length' : [], 'Magnetization_mean':[], 'Magnetization_min':[], 'Magnetization_max':[],
        'Magnetization_std':[], 'SpinGlass':[], 'unstables':[]}   
conf = Ring.createRing(n)

In [92]:
for i, mag in enumerate(magnetList):
    print(i)
    for j in range(samples):
        init_states= [State.ON]*round((size*(0.5 + mag/2))-10**(-9)) + [State.OFF]*round((size*(0.5 -mag/2)+10**(-9)))
        np.random.shuffle(init_states)
        init_rule = np.array([Rule.STABLE]*size)
        curr_rule =init_rule
        conf.set_rules(init_rule)
        conf.set_states(init_states)
        curr_magnet = mag
        stables = [0]
        while curr_magnet<1 and stables != []:
            array, out = Evolve.evolve(conf, perm= pi, steps=steps, 
                                   metricList=metricList, cycleBreak=True)
            curr_energy = out[Metric.SpinGlass][-1]
            curr_magnet= out[Metric.Magnetization][-1]
            length, start = utilities.cycle_length(array)

            stables = [k for k in range(n) if curr_rule[k]== Rule.STABLE]

            agree['init_magnet'].append(mag)
            agree['length'].append(length)
            agree['Magnetization_mean'].append(curr_magnet)
            agree['Magnetization_std'].append(np.std(magnetization(array[start: start+length])))
            agree['Magnetization_min'].append(np.min(magnetization(array[start: start+length])))
            agree['Magnetization_max'].append(np.max(magnetization(array[start: start+length])))
            agree['SpinGlass'].append(curr_energy)  
            agree['unstables'].append(n-len(stables))

            # Choose a random stable node to change rule
            index = 0 if stables == [] else np.random.choice(stables)
            curr_rule[index] = Rule.UNSTABLE
            conf.set_rules(curr_rule)
data = pd.DataFrame(agree)


0
1
2


In [93]:
title = '{2}_{0}_{1}_agreement_multi.csv'.format(n, steps, c_type.name)
data.to_csv(title, encoding='utf-8', index=False)


### Load

In [8]:
#n=100
#steps=1000
title = '{2}_{0}_{1}_agreement_multi.csv'.format(n, steps, c_type.name)
data= pd.read_csv(data_dir /title)

### Magnetization obtained according to stability

In [9]:
data

,init_magnet,length,Magnetization_mean,Magnetization_min,Magnetization_max,Magnetization_std,SpinGlass,unstables
0,0.0,1,0.06,0.06,0.06,0.00000,-72.0,0
1,0.0,2,0.05,0.04,0.06,0.01000,-72.0,1
2,0.0,2,0.05,0.04,0.06,0.01000,-72.0,2
3,0.0,1,0.00,0.00,0.00,0.00000,-72.0,3
4,0.0,2,0.01,0.00,0.02,0.01000,-72.0,4
...,...,...,...,...,...,...,...,...
80774,0.8,15,0.90,0.84,0.96,0.03266,-96.0,39
80775,0.8,15,0.90,0.84,0.96,0.03266,-96.0,40
80776,0.8,15,0.90,0.84,0.96,0.03266,-96.0,41
80777,0.8,1,1.00,1.00,1.00,0.00000,-100.0,42


In [15]:
facet = sns.FacetGrid(data,  # the dataframe to pull from
                          row="init_magnet",  # define the column for each subplot row to be differentiated by
                          hue="init_magnet",  # define the column for each subplot color to be differentiated by
                          aspect=6,  # aspect * height = width
                          height=2,  # height of each subplot
                          )
by_magnet = data.groupby("init_magnet")
def init_magnet(x, color, label):
    ax= plt.gca()
    fix_magnet = by_magnet.get_group(float(label))
    val = np.mean(fix_magnet.loc[fix_magnet["unstables"] == 0]["Magnetization_mean"].values)
    print(label, val)
    plt.axhline(y=val, ls="--")
    
facet.map(init_magnet, "init_magnet")
facet.map(sns.lineplot,'unstables',"Magnetization_mean")
#facet.map(sns.lineplot, 'unstables', "Magnetization_min", ls="--")
#facet.map(sns.lineplot, 'unstables', "Magnetization_max", ls= "--")

def label(x, color, label):
    ax = plt.gca()  # get the axes of the current object
    ax.text(0, .2,  # location of text
            label,  # text label
            fontweight="bold", color=color, size=18,  # text attributes
            ha="left", va="center",  # alignment specifications
            transform=ax.transAxes)  # specify axes of transformation
        

facet.map(label, "Magnetization_mean")  # the function counts as a plotting object!
facet.set_titles("")  # set title to blank

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.0 0.06939999999999999
0.2 0.3001
0.4 0.5743
0.6 0.7927000000000001
0.8 0.9494999999999999


In [16]:
facet = sns.FacetGrid(data,  # the dataframe to pull from
                          row="init_magnet",  # define the column for each subplot row to be differentiated by
                          hue="init_magnet",  # define the column for each subplot color to be differentiated by
                          aspect=6,  # aspect * height = width
                          height=2,  # height of each subplot
                          )
def init_magnet(x, color, label):
    ax= plt.gca()
    fix_magnet = by_magnet.get_group(float(label))
    val = np.mean(fix_magnet.loc[fix_magnet["unstables"] == 0]["SpinGlass"].values)
    print(label, val)
    plt.axhline(y=val, ls="--")
    
facet.map(init_magnet, "init_magnet")
facet.map(sns.lineplot,'unstables',"SpinGlass")
#facet.map(sns.lineplot, 'unstables', "Magnetization_min", ls="--")
#facet.map(sns.lineplot, 'unstables', "Magnetization_max", ls= "--")

def label(x, color, label):
    ax = plt.gca()  # get the axes of the current object
    ax.text(0, .2,  # location of text
            label,  # text label
            fontweight="bold", color=color, size=18,  # text attributes
            ha="left", va="center",  # alignment specifications
            transform=ax.transAxes)  # specify axes of transformation
        

facet.map(label, "SpinGlass")  # the function counts as a plotting object!
facet.set_titles("")  # set title to blank

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.0 -63.24
0.2 -65.86
0.4 -74.2
0.6 -84.96
0.8 -95.86


In [80]:
by_magnet = data.groupby("init_magnet")

In [85]:
fix_magnet = by_magnet.get_group(0.6)
val = np.mean(fix_magnet.loc[fix_magnet["unstables"] == 0]["Magnetization_mean"].values)
val

0.7897999999999997

In [17]:
facet = sns.FacetGrid(data,  # the dataframe to pull from
                          row="init_magnet",  # define the column for each subplot row to be differentiated by
                          hue="init_magnet",  # define the column for each subplot color to be differentiated by
                          aspect=6,  # aspect * height = width
                          height=2,  # height of each subplot
                          )
def init_magnet(x, color, label):
    ax= plt.gca()
    fix_magnet = by_magnet.get_group(float(label))
    val = np.mean(fix_magnet.loc[fix_magnet["unstables"] == 0]["SpinGlass"].values)
    print(label, val)
    plt.axhline(y=val, ls="--")
    
#facet.map(init_magnet, "init_magnet")
facet.map(sns.lineplot,'unstables',"length")
#facet.map(sns.lineplot, 'unstables', "Magnetization_min", ls="--")
#facet.map(sns.lineplot, 'unstables', "Magnetization_max", ls= "--")

def label(x, color, label):
    ax = plt.gca()  # get the axes of the current object
    ax.text(0, .2,  # location of text
            label,  # text label
            fontweight="bold", color=color, size=18,  # text attributes
            ha="left", va="center",  # alignment specifications
            transform=ax.transAxes)  # specify axes of transformation
        

facet.map(label, "length")  # the function counts as a plotting object!
facet.set_titles("")  # set title to blank

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
def init_magnet(x, color, label):
    ax= plt.gca()
    fix_magnet = by_magnet.get_group(float(label))
    val = fix_magnet.loc[fix_magnet["unstables"] == 0]["Magnetization_mean"].values[0]
    plt.axhline(y=val, ls="--")
    
facet.map(init_magnet, "init_magnet")
#facet.map(sns.lineplot,'unstables',"Magnetization_mean")
facet.map(sns.lineplot, 'unstables', "Magnetization_min", ls="--")
facet.map(sns.lineplot, 'unstables', "Magnetization_max", ls= "--")

0.0
0.2
0.4
0.6
0.8
